In [ ]:
#set cores for parallel computing
RNGkind("L'Ecuyer-CMRG")
cores<-strtoi(Sys.getenv('SLURM_CPUS_PER_TASK', unset=1))
registerDoMC(cores)

part=1

res<-foreach(i=1:1) %dopar% {
  cat("i= ",i,'\n')

finished <- FALSE
while(!finished) {
    tryCatch({

  ##########################################
  #    simulate data for linear case       #
  ##########################################
  ## simulate simple data with one x
  beta0=log(2)
  beta1=0.2
  sigmav=0.9
  sigmau=0.2
  nsamp=200

  x_org=simulated_datasets_R[[i]]$log_X
  y_org=simulated_datasets_R[[i]]$log_y


  ########################################
  # Model 1 fit the Bayes Linear Model
  ########################################

  simdt=data.frame(y1=y_org,lnx1=x_org)

  B_burnin=1000
  B_afterburnin=4000

  # B_burnin=2
  # B_afterburnin=20

  res_Bayelin<-Bayeslinear_NHN_fit(B_burnin,
                                   B_afterburnin,
                                   simdt,
                                   beta0_init = 0,
                                   beta_init=beta0,
                                   sigu_init=sigmau,
                                   sigv_init=sigmav,
                                   df_v = 5,
                                   S_v =(5+2)*sigmav^2,
                                   df_u =5,
                                   S_u = (5+2)*sigmau^2)


  B=B_burnin+B_afterburnin
  res_bayes_lin<-na.omit(res_Bayelin$res[B_burnin+1:B,])

  ##############################################
  # Model 2
  # fit SEM-SFM
  # imposing monotonicity restrictions on inputs
  ##############################################
  #first-step: monotone gam, second-step: fan
  dati=data.frame(y=y_org,x=x_org)
  semfit<-semsfa(y~pbm(x,mono="up"),sem.method = "gam.mono",dati)
  sigma_v_semfit=sqrt(semfit$sigma^2/(1+semfit$lambda^2))
  sigma_u_semfit=semfit$lambda*sigma_v_semfit

  ##############################################
  # Model 3
  # fit Mon-BART-SFM
  # imposing monotonicity restrictions on inputs
  ##############################################
  #find initial value for sigma_v and sigma_u

  fit_sfa <-sfa( y1 ~ lnx1,data = simdt)
  gamma <- unname( coef(fit_sfa)["gamma"] )
  sigmaSq <- unname( coef(fit_sfa)["sigmaSq"] )
  sigmaSqU <- gamma * sigmaSq
  sigmaSqV <- ( 1 - gamma ) * sigmaSq
  betavec<-coef(fit_sfa)[1:3]
  sigma_u<-sqrt(sigmaSqU)
  sigma_v<-sqrt(sigmaSqV)



  ysnfit<-selm(y_org ~ 1, family="SN")
  center_Y <- ysnfit@param$dp[1]

  nskip=B_burnin
  ndpost=B_afterburnin


  fitbartsfm<-monbartsfm(
    x.train=as.matrix(x_org),y.train=y_org, x.test=matrix(0.0,0,0),
    sigest=sigma_v,
    sigdf=3,
    sigquant=.95,
    sigu_pre_est=sigma_u,
    sigquant_u=.95,
    k=2.0,
    power=.8, base=.25, #regular bart is 2,.95
    sigmaf=NA,
    lambda=NA,
    fmean = center_Y,
    #fscale=scale_Y,
    ntree=100,
    ndpost, nskip,
    mgsize=50,
    nkeeptrain=ndpost,nkeeptest=ndpost,
    nkeeptestmean=ndpost,
    nkeeptreedraws=ndpost,
    printevery=100
  )



  fitbartsfm_beta0<-monbartsfm_beta0(
    x.train=as.matrix(x_org),y.train=y_org, x.test=matrix(0.0,0,0),
    sigest=sigma_v,
    sigdf=3,
    sigquant=.95,
    sigu_pre_est=sigma_u,
    sigquant_u=.95,
    k=2.0,
    power=.8, base=.25, #regular bart is 2,.95
    sigmaf=NA,
    lambda=NA,
    fmean = center_Y,
    #fscale=scale_Y,
    ntree=100,
    ndpost, nskip,
    mgsize=50,
    nkeeptrain=ndpost,nkeeptest=ndpost,
    nkeeptestmean=ndpost,
    nkeeptreedraws=ndpost,
    printevery=100
  )

  gtest<-geweke.diag(fitbartsfm$sigma_u)
  pval_BART<-1-pnorm(gtest$z)
  # pval_BART=0.5
  gtest_b0<-geweke.diag(fitbartsfm_beta0$sigma_u)
  pval_BART_b0<-1-pnorm(gtest_b0$z)
  # pval_BART_b0=0.5
  finished <- TRUE #if convergence test is passed, then next iteration

  if(pval_BART>0.1&pval_BART_b0>0.1){
    finished <- TRUE #if convergence test is passed, then next iteration
  }

  #######################################
  # 1. Compare RMSE and BIAS            #
  #######################################

  y_tf = exp(beta0)*exp(x_org)^beta1 #((X**w_true)*b_true).numpy() 
  #############################################################################
  #formulas are from Eq. 10 and 11 from                                       #
  #Ferrara and Vidoli 2017 Semiparametric Stochastic frontier models          #
  #############################################################################
    
  RMSE_bayeslin           = 0 #mean(((mean(res_Bayelin$res$beta0_post)+mean(res_Bayelin$res$beta_current)*x_org-y_org)/y_org)^2) # I did not calculate the Bayes lin
  e_sem = y_org - (semfit$fitted)
  E_sem_correction = mean(exp(e_sem))
  RMSE_sem                =mean(((exp(semfit$fitted)*E_sem_correction-y_tf)/y_tf)^2)
  e_bartsfm = y_org - (fitbartsfm$yhat.train.mean)
  E_bartsfm_correction = mean(exp(e_bartsfm))
  RMSE_bartsfm            =mean(((exp(fitbartsfm$yhat.train.mean)*E_bartsfm_correction-y_tf)/y_tf)^2)
  # RMSE_bartsfm_uadj       =mean(((fitbartsfm_uadj$yhat.train.mean-y_org)/y_org)^2)
  e_bartsfm_beta0 = y_org - (fitbartsfm_beta0$yhat.train.mean)
  E_bartsfm_beta0_correction = mean(exp(e_bartsfm_beta0))
  RMSE_bartsfm_beta0      =mean(((exp(fitbartsfm_beta0$yhat.train.mean)*E_bartsfm_beta0_correction-y_tf)/y_tf)^2)
  # RMSE_bartsfm_beta0_uadj =mean(((fitbartsfm_beta0_uadj$yhat.train.mean-y_org)/y_org)^2)

  BIAS_bayeslin           =mean(abs((mean(res_Bayelin$res$beta0_post)+mean(res_Bayelin$res$beta_current)*x_org-y_org)/y_org))
  BIAS_sem                =mean(abs((exp(semfit$fitted)*E_sem_correction-y_tf)/y_tf))
  BIAS_bartsfm            =mean(abs((exp(fitbartsfm$yhat.train.mean)*E_bartsfm_correction-y_tf)/y_tf))
  # BIAS_bartsfm_uadj       =mean(((fitbartsfm_uadj$yhat.train.mean-y_org)/y_org))
  BIAS_bartsfm_beta0      =mean(abs((exp(fitbartsfm_beta0$yhat.train.mean)*E_bartsfm_beta0_correction-y_tf)/y_tf))
  # BIAS_bartsfm_beta0_uadj =mean(((fitbartsfm_beta0_uadj$yhat.train.mean-y_org)/y_org))

  RMSE_res<-c(RMSE_bayeslin,
              RMSE_sem,
              RMSE_bartsfm,
            # RMSE_bartsfm_uadj,
              RMSE_bartsfm_beta0
  )

  BIAS_res <- c(BIAS_bayeslin,
                BIAS_sem,
                BIAS_bartsfm,
                # BIAS_bartsfm_uadj,
                BIAS_bartsfm_beta0
                # BIAS_bartsfm_beta0_uadj
                )

  ##############################################################################
  # 2. Compare mean bias estimates of sigma_u and sigma_v                           #
  ##############################################################################
  Bias_sig_v_bayeslin           = abs(mean(res_bayes_lin$sigv_post)-sigmav)
  Bias_sig_v_sem                = abs(sigma_v_semfit-sigmav)
  Bias_sig_v_bartsfm            = abs(mean(fitbartsfm$sigma[(nskip+1):(nskip+ndpost)])      -sigmav)
  # Bias_sig_v_bartsfm_uadj       = abs(mean(fitbartsfm_uadj$sigma[(nskip+1):(nskip+ndpost)]) -sigmav)
  Bias_sig_v_bartsfm_beta0      = abs(mean(fitbartsfm_beta0$sigma[(nskip+1):(nskip+ndpost)])-sigmav)
  # Bias_sig_v_bartsfm_beta0_uadj = abs(mean(fitbartsfm_uadj$sigma[(nskip+1):(nskip+ndpost)]) -sigmav)

  Bias_sig_u_bayeslin           = abs(mean(res_bayes_lin$sigu_post)-sigmau)
  Bias_sig_u_sem                = abs(sigma_u_semfit-sigmau)
  Bias_sig_u_bartsfm            = abs(mean(fitbartsfm$sigma_u[(nskip+1):(nskip+ndpost)])     -sigmau)
  # Bias_sig_u_bartsfm_uadj       = abs(mean(fitbartsfm_uadj$sigma_u[(nskip+1):(nskip+ndpost)])  -sigmau)
  Bias_sig_u_bartsfm_beta0      = abs(mean(fitbartsfm_beta0$sigma_u[(nskip+1):(nskip+ndpost)])  -sigmau)
  # Bias_sig_u_bartsfm_beta0_uadj = abs(mean(fitbartsfm_uadj$sigma_u[(nskip+1):(nskip+ndpost)])   -sigmau)

  Bias_sig_v_res <- c(
      Bias_sig_v_bayeslin      ,
      Bias_sig_v_sem           ,
      Bias_sig_v_bartsfm       ,
      # Bias_sig_v_bartsfm_uadj  ,
      Bias_sig_v_bartsfm_beta0
      # Bias_sig_v_bartsfm_beta0_uadj
    )

  Bias_sig_u_res <-
    c(Bias_sig_u_bayeslin      ,
      Bias_sig_u_sem           ,
      Bias_sig_u_bartsfm       ,
      # Bias_sig_u_bartsfm_uadj  ,
      Bias_sig_u_bartsfm_beta0
      # Bias_sig_u_bartsfm_beta0_uadj
    )


  ##############################################################################
  # 3. Compare mean inefficiency scores                                        #
  ##############################################################################
  #True TEs
  res_true<-simdt$y1-(beta0+beta1*simdt$lnx1)
  TEvec_true<-sapply(res_true,TE_fun,sigv=sigmav,sigu=sigmau)
  mean(TEvec_true)

  #Bayes regression TEs
  TEvec_bayeslin<-apply(res_bayes_lin,1,function(x){ return(TE_bayes_fun(bayelinres = x,y = simdt$y1,x=simdt$lnx1))})
  mean(TEvec_bayeslin)

  #TE for sem
  res_sem<-simdt$y1-semfit$fitted
  TEvec_sem<-sapply(res_sem,TE_fun,sigv=sigma_v_semfit,sigu=sigma_u_semfit)
  mean(TEvec_sem)

  #TE for bart-sfm
  TEvec_bartsfm<-apply(cbind(fitbartsfm$yhat.train,
                             fitbartsfm$sigma[(nskip+1):(nskip+ndpost)],
                             fitbartsfm$sigma_u[(nskip+1):(nskip+ndpost)]),1,
                       function(y){return(TE_bart_fun(yhat_post = y[1:nsamp],
                                                      sigv_post = y[nsamp+1],
                                                      sigu_post = y[nsamp+2],
                                                      y_bart = simdt$y1
                       ))})

  #TE for bart-sfm-uadj
  # TEvec_bartsfm_uadj<-apply(cbind(fitbartsfm_uadj$yhat.train,
  #                                 fitbartsfm_uadj$sigma[(nskip+1):(nskip+ndpost)],
  #                                 fitbartsfm_uadj$sigma_u[(nskip+1):(nskip+ndpost)]),1,
  #                           function(y){return(TE_bart_fun(yhat_post = y[1:nsamp],
  #                                                          sigv_post = y[nsamp+1],
  #                                                          sigu_post = y[nsamp+2],
  #                                                          y_bart = simdt$y1
  #                           ))})

  #TE for bart-sfm-beta0
  TEvec_bartsfm_beta0<-apply(cbind(fitbartsfm_beta0$yhat.train,
                                   fitbartsfm_beta0$sigma[(nskip+1):(nskip+ndpost)],
                                   fitbartsfm_beta0$sigma_u[(nskip+1):(nskip+ndpost)]),1,
                             function(y){return(TE_bart_fun(yhat_post = y[1:nsamp],
                                                            sigv_post = y[nsamp+1],
                                                            sigu_post = y[nsamp+2],
                                                            y_bart = simdt$y1
                             ))})


  #TE for bart-sfm-beta0-uadj
  # TEvec_bartsfm_beta0_uadj<-apply(cbind(fitbartsfm_beta0_uadj$yhat.train,
  #                                       fitbartsfm_beta0_uadj$sigma[(nskip+1):(nskip+ndpost)],
  #                                       fitbartsfm_beta0_uadj$sigma_u[(nskip+1):(nskip+ndpost)]),1,
  #                                 function(y){return(TE_bart_fun(yhat_post = y[1:nsamp],
  #                                                                sigv_post = y[nsamp+1],
  #                                                                sigu_post = y[nsamp+2],
  #                                                                y_bart = simdt$y1
  #                                 ))})

  TE_bias<-abs(c(
    mean(TEvec_bayeslin),
    mean(TEvec_sem),
    mean(TEvec_bartsfm),
    # mean(TEvec_bartsfm_uadj),
    mean(TEvec_bartsfm_beta0)
    # mean(TEvec_bartsfm_beta0_uadj)
  )-mean(TEvec_true))

    },
  error = function(e) {})
}

  # TE_bias<-rbind(TE_bias,TE_bias_one)

  return(list(RMSE=RMSE_res,
              BIAS=BIAS_res,
              Bias_sig_v=Bias_sig_v_res,
              Bias_sig_u=Bias_sig_u_res,
              TE_bias=TE_bias,
              simdata=simdt,
              sig_v_bart_post=fitbartsfm$sigma,
              sig_u_bart_post=fitbartsfm$sigma_u,
              sig_v_bart_b0_post=fitbartsfm_beta0$sigma,
              sig_u_bart_b0_post=fitbartsfm_beta0$sigma_u
              ))

}

RMSE_res<-as.data.frame(do.call(rbind,lapply(res,function(x){x[[1]]})))
colnames(RMSE_res)<-c("RMSE_BayesLin","RMSE_Sem",
                      "RMSE_Bart",
                      # "RMSE_Bart_uadj",
                      "RMSE_Bart_b0"
                      # "RMSE_Bart_uadj_b0"
)

BIAS_res<-as.data.frame(do.call(rbind,lapply(res,function(x){x[[2]]})))
colnames(BIAS_res)<-c("BIAS_BayesLin","BIAS_Sem",
                      "BIAS_Bart",
                      # "BIAS_Bart_uadj",
                      "BIAS_Bart_b0"
                      # "BIAS_Bart_uadj_b0"
)

Bias_sig_v_res<-as.data.frame(do.call(rbind,lapply(res,function(x){x[[3]]})))
colnames(Bias_sig_v_res)<-c("Sigvb_BayesLin","Sigvb_Sem",
                            "Sigvb_Bart",
                            # "Sigvb_Bart_uadj",
                            "Sigvb_Bart_b0"
                            # "Sigvb_Bart_uadj_b0"
)

Bias_sig_u_res<-as.data.frame(do.call(rbind,lapply(res,function(x){x[[4]]})))
colnames(Bias_sig_u_res)<-c("Sigub_BayesLin",
                            "Sigub_Sem",
                            "Sigub_Bart",
                            # "Sigub_Bart_uadj",
                            "Sigub_Bart_b0"
                            # "Sigub_Bart_uadj_b0"
)

TE_bias<-as.data.frame(do.call(rbind,lapply(res,function(x){x[[5]]})))
colnames(TE_bias)<-c("TEb_BayesLin","TEb_Sem",
                     "TEb_Bart",
                     # "TEb_Bart_uadj",
                     "TEb_Bart_b0"
                     # "TEb_Bart_uadj_b0"
)

resfin<-list(RMSE=RMSE_res,
             BIAS=BIAS_res,
             Bias_sigv=Bias_sig_v_res,
             Bias_sigu=Bias_sig_u_res,
             TE_bias=TE_bias,
             res=res)
saveRDS(resfin, paste("res/linear_case_part",part,".rds",sep = ""))